In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import fxcmpy

In [2]:
con = fxcmpy.fxcmpy(config_file = 'fxcm.cfg', log_level='debug', server='demo')

In [15]:
con.debug = True

In [16]:
con.is_connected()

True

In [25]:
con.get_max_prices()

10000

In [95]:
poses = con.get_open_positions()
print(poses.T)
for index, pos in poses.iterrows():
    print(con.close_trade(int(pos.tradeId)))

                            0
accountId             1041561
accountName          01041561
amountK                     1
close                 1.12358
com                         0
currency              EUR/USD
currencyPoint             0.1
grossPL                 -0.17
isBuy                    True
isDisabled              False
limit                       0
open                  1.12375
ratePrecision               5
roll                        0
stop                        0
stopMove                    0
t                           1
time           04192019041021
tradeId              67770991
usedMargin             21.645
valueDate                    
visiblePL                -1.7


ValueError: No open position with id 67770991.

In [ ]:
closes = con.get_closed_positions()
print(closes.T)
for i, close in closes.iterrows():
    print(con.get_closed_position(int(close.tradeId)))

Exception in thread Thread-24:
Traceback (most recent call last):
  File "/anaconda2/envs/fxcm/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/anaconda2/envs/fxcm/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/fxcmpy/fxcmpy.py", line 2364, in __connect__
    self.socket.wait()
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/socketIO_client/__init__.py", line 251, in wait
    self._process_packets()
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/socketIO_client/__init__.py", line 278, in _process_packets
    self._process_packet(engineIO_packet)
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/socketIO_client/__init__.py", line 491, in _process_packet
    delegate(parse_socketIO_packet_data(socketIO_packet_data), namespace)
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/socketIO_client/__init__.py", line 513, in _on

In [97]:
con.get_order(67770991)

ValueError: No order with id 67770991

In [93]:
def test():
    print('received data')
    
con.subscribe_market_data(symbol='EUR/USD', add_callbacks=[test])

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/anaconda2/envs/fxcm/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/anaconda2/envs/fxcm/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/fxcmpy/fxcmpy.py", line 2364, in __connect__
    self.socket.wait()
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/socketIO_client/__init__.py", line 251, in wait
    self._process_packets()
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/socketIO_client/__init__.py", line 278, in _process_packets
    self._process_packet(engineIO_packet)
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/socketIO_client/__init__.py", line 491, in _process_packet
    delegate(parse_socketIO_packet_data(socketIO_packet_data), namespace)
  File "/anaconda2/envs/fxcm/lib/python3.7/site-packages/socketIO_client/__init__.py", line 513, in _on

In [3]:
con.get_summary().T

,0
amountK,-1
amountKBuy,0
amountKSell,1
avgBuy,0
avgSell,1.12442
closeBuy,1.12461
closeSell,0
currency,EUR/USD
currencyPoint,-0.1
grossPL,-0.19


In [5]:
poses = con.get_open_positions_summary()
for i, pos in poses.iterrows():
    print(pos.grossPL)

-0.16


In [49]:
con.get_closed_positions_summary().T

,0
accountName,
amountK,1
close,0
closeTime,None
com,0
currency,EUR/USD
currencyPoint,0
grossPL,-0.36
isBuy,False
isTotal,True


In [42]:
con.get_accounts().T

,0
accountId,1041561
accountName,01041561
balance,39967.4
dayPL,0.63
equity,39967.3
grossPL,-0.16
hedging,Y
mc,N
mcDate,
ratePrecision,0


In [54]:
con.get_order_ids()

[]

In [53]:
con.get_open_trade_ids()

[]

In [55]:
con.get_closed_trade_ids()

[]

In [56]:
con.get_all_trade_ids()

[]

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import fxcmpy
from apscheduler.schedulers.blocking import BlockingScheduler
import os

os_path = os.getcwd()

def z(df):
    return (df - df.mean()) / df.std()

sched = BlockingScheduler()
lot = 100
tf = 'm5'
con = fxcmpy.fxcmpy(config_file = 'fxcm.cfg')

@sched.scheduled_job('interval', seconds=300)
def timed_job():
    print('new time job')
    if (len(con.get_open_positions()) > 0):
        check_position(con)
    trading(con)
    
def check_position(con):
    i = 0
    for i in range(len(con.open_pos)):
        #print('i = %i' % i)
        trade_id = con.get_open_trade_ids()[i]
        pos = con.get_open_position(trade_id)
        amount = pos.get_amount()
        symbol = pos.get_currency()
        data = con.get_candles(symbol, period=tf, number=100)
        if (pos.get_isBuy()):
            data['Close_z'] = z(data['bidclose'])
            #print(data.iloc[-1]['Close_z'])
            if data.iloc[-1]['Close_z'] > -1:
                con.close_trade(trade_id=trade_id, amount=amount)
                print('close %s' % symbol)
            else:
                #print('no close')
        else:
            data['Close_z'] = z(data['askclose'])
            #print(data.iloc[-1]['Close_z'])
            if data.iloc[-1]['Close_z'] < 1:
                con.close_trade(trade_id=trade_id, amount=amount)
                print('close %s id: %i amount: %i' % (symbol, trade_id, amount))
            else:
                #print('no close')
        i += 1
            

    
def trading(con):
    
    symbols = ['EUR/USD', 'USD/JPY', 'GBP/USD', 'USD/CAD', 'USD/CHF', 'AUD/USD', 'NZD/USD', 'EUR/AUD', 'EUR/CAD', 'EUR/CHF', 'EUR/GBP', 'EUR/JPY', 'EUR/NZD', 'GBP/AUD', 'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'GBP/NZD', 'AUD/CAD', 'AUD/CHF', 'AUD/JPY', 'AUD/NZD', 'NZD/CAD', 'NZD/CHF', 'NZD/JPY', 'CAD/CHF', 'CAD/JPY', 'CHF/JPY']
    df = dict()
    for symbol in symbols:
        df[symbol] = con.get_candles(symbol, period=tf, number=100)
        df[symbol]['bid_Close_z'] = z(df[symbol]['bidclose'])
        df[symbol]['ask_Close_z'] = z(df[symbol]['askclose'])
        if df[symbol].iloc[-1]['ask_Close_z'] < -2:
            print('buy signal')
            if not hasCurrency(con, symbol):
                order = con.create_market_buy_order(symbol, lot)
                if order:
                    print('buy %s success' % symbol)
                else:
                    print('buy %s not success' % symbol)
        elif df[symbol].iloc[-1]['bid_Close_z'] > 2:
            print('sell signal')
            if not hasCurrency(con, symbol):
                order = con.create_market_sell_order(symbol, lot)
                if order:
                    print('sell %s success' % symbol)
                else:
                    print('sell %s not success' % symbol)
    print('---------')

def hasCurrency(con, symbol):
    i = 0
    for i in range(len(con.open_pos)):
        trade_id = con.get_open_trade_ids()[i]
        pos = con.get_open_position(trad5e_id)
        if symbol == pos.get_currency():
            print('have position')
            return True
    print('%i not have position %s' % (i, symbol))
    return False
    

#@sched.scheduled_job('cron', day_of_week='mon-fri', hour=10)
#def scheduled_job():
#    print('This job is run every weekday at 10am.')


In [2]:
if __name__ == "__main__":  
    sched.start()

new time job
i = 0
-0.6443965279774007
close EUR/JPY
i = 1
0.6341650187668785
close USD/JPY
i = 2
-1.138809992579526
no close
i = 3
new time job
i = 0
-0.17480932561380164
close EUR/GBP
i = 1
-0.9180360613915809
close EUR/USD
sell signal
0 not have position AUD/CHF
sell AUD/CHF success
sell signal
new time job
i = 0
2.0328834371379427
no close
sell signal
new time job
i = 0
1.2774801581352855
no close
---------
new time job
i = 0
0.6139737488777671
close AUD/CHF id: 67575515 amount: 100
buy signal
0 not have position EUR/CHF
buy EUR/CHF success
---------
new time job
i = 0
-2.7983437431150415
no close
buy signal
new time job
i = 0
-2.4099069876670014
no close
buy signal
new time job
i = 0
-2.092246696273213
no close
buy signal
new time job
i = 0
-1.9360090695332501
no close
---------
new time job
i = 0
-1.9531648469891143
no close
---------
new time job
i = 0
-1.6485849120828286
no close
---------
new time job
i = 0
-1.8091738249914937
no close
buy signal
new time job
i = 0
-2.04023626

KeyboardInterrupt: 

In [2]:
os_path

'/Users/ballmdr/blockchain-python/FXCM/Python Trading'